In [66]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up the scope for the APIs
scope = ["https://www.googleapis.com/auth/spreadsheets.readonly", 
         "https://www.googleapis.com/auth/drive.readonly"]

# Provide the path to the credentials JSON file you downloaded
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/alexandercappelen/Documents/keys/frb-elite-88e4dcc7ec5c.json', scope)

# Authorize and create the client
client = gspread.authorize(creds)

# Open the sheet by name
player_earned = client.open("frb-volley-game-stats").worksheet("player-offense")

# Fetch all records (rows) from the sheet
player_earned_data = player_earned.get_all_records()



In [67]:
# Print the data or convert it into a DataFrame
import pandas as pd
df = pd.DataFrame(player_earned_data)

df["error-pct"] = df["attack-errors"] / df["attack"]
df

,date,match-id,match,set,player,serves,aces,serve-error,points-won,point-scoring-pct,attack,kills,attack-errors,kill-pct,kill-effic,error-pct
0,08/12/2024,141108,frb-gentofte,1,Lasse,2,1,0,1,50.0,1,1,0,100.000000,1.000000,0.000000
1,08/12/2024,141108,frb-gentofte,1,Nico,0,0,0,0,0.0,0,0,0,0.000000,0.000000,NaN
2,08/12/2024,141108,frb-gentofte,1,Kristian,5,0,2,3,60.0,2,1,0,50.000000,0.500000,0.000000
3,08/12/2024,141108,frb-gentofte,1,Ando,0,0,0,0,0.0,0,0,0,0.000000,0.000000,NaN
4,08/12/2024,141108,frb-gentofte,1,Bosse,0,0,0,0,0.0,0,0,0,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,29/09/2024,141063,gentofte-frb,5,Mads,0,0,0,0,0.0,0,0,0,0.000000,0.000000,NaN
328,29/09/2024,141063,gentofte-frb,5,Martin,1,0,1,0,0.0,7,5,1,71.428571,0.571429,0.142857
329,29/09/2024,141063,gentofte-frb,5,Mikkel,0,0,0,0,0.0,0,0,0,0.000000,0.000000,NaN
330,29/09/2024,141063,gentofte-frb,5,Frederik,0,0,0,0,0.0,0,0,0,0.000000,0.000000,NaN


In [69]:
summary = df.groupby('player').agg(
    attack_attempts=('attack', 'sum'),
    total_kills=('kills', 'sum'),
    attack_errors=('attack-errors', 'sum')
).reset_index()

summary["kill_pct"] = ((summary["total_kills"]) / summary["attack_attempts"] * 100).round(0)
summary["error_pct"] = ((summary["attack_errors"]) / summary["attack_attempts"] * 100).round(0)
summary["kill_effic"] = ((summary["total_kills"] - summary["attack_errors"]) / summary["attack_attempts"])#.round(3)
summary = summary.dropna()
summary["kill_pct"] = summary["kill_pct"].astype(int)
summary["error_pct"] = summary["error_pct"].astype(int)
summary["kill_effic"] = summary["kill_effic"].round(3)
summary

,player,attack_attempts,total_kills,attack_errors,kill_pct,error_pct,kill_effic
2,Bo,23,9,3,39,13,0.261
3,Boerme,91,37,15,41,16,0.242
6,Gustav,39,10,5,26,13,0.128
7,Kristian,30,11,2,37,7,0.300
8,Lasse,33,16,2,48,6,0.424
9,Mads,1,1,0,100,0,1.000
10,Martin,89,38,14,43,16,0.270
12,Nico,1,0,0,0,0,0.000
13,Nicola,142,60,21,42,15,0.275
14,Soeren,112,38,16,34,14,0.196


In [70]:
import json

# Convert DataFrame to JSON
data = summary.to_dict(orient="records")  # Convert DataFrame rows to list of dictionaries
with open("../player-offense-summary.json", "w") as f:
    json.dump(data, f, indent=4)